# ODE Modeling of Neuroblast Lineage Growth

In [ ]:
from scipy.integrate import solve_ivp
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

# Font size settings for figures
mpl.rcParams.update({
    "font.size": 14,              # base font size
    "axes.titlesize": 16,         # title font
    "axes.labelsize": 14,         # x/y label font
    "xtick.labelsize": 12,        # x-tick labels
    "ytick.labelsize": 12,        # y-tick labels
    "legend.fontsize": 12,        # legend text
    "lines.linewidth": 2,         # line thickness
    "axes.linewidth": 1.2,        # axis border thickness
})

# Function to make annotating data on figures easier
def annotate_final_values(ax, x, y_values, labels, min_sep_frac=0.1):
    """
    Adds annotations at the final time point of each y-series with spacing done in axis-fraction units.

    Parameters:
        ax: matplotlib Axes object
        x: time points (1D array)
        y_values: list of y-arrays
        labels: list of label strings
        min_sep_frac: minimum spacing between annotations in axis fraction (0.03 = 3% of y-axis height)
    """
    final_x = x[-1]
    final_ys = [y[-1] for y in y_values]
    y_min, y_max = ax.get_ylim()

    # Convert y-values to axis-fraction space
    y_fracs = [(y - y_min) / (y_max - y_min) for y in final_ys]

    # Sort and apply minimum spacing in axis-fraction space
    sorted_indices = np.argsort(y_fracs)
    new_frac_positions = {}
    last_frac = None
    for idx in sorted_indices:
        frac = y_fracs[idx]
        if last_frac is None:
            new_frac_positions[idx] = frac
        else:
            new_frac_positions[idx] = max(frac, last_frac + min_sep_frac)
        last_frac = new_frac_positions[idx]

    # Clamp to [0,1] and convert back to data coordinates
    for idx in range(len(y_values)):
        orig_y = final_ys[idx]
        frac_y = min(new_frac_positions[idx], 0.98)  # avoid going out of bounds
        adjusted_y = y_min + frac_y * (y_max - y_min)

        ax.annotate(f"{labels[idx]}\n{orig_y:.0f}",
                    xy=(final_x, orig_y),
                    xytext=(final_x + 1, adjusted_y),
                    textcoords='data',
                    fontsize=8,
                    va='center',
                    ha='left',
                    arrowprops=dict(arrowstyle='->', lw=0.5),
                    bbox=dict(boxstyle="round,pad=0.2", fc="white", ec="gray", lw=0.5))

<a id="outline"></a>
## Notebook Outline:
- [Modeling Cell Counts](#cellcounts)
- [Modeling Cell Counts with Activating Cell Count Feedback on Neuron Differentiation Term](#cellcountsneurondiffcellcount)
- [Modeling Cell Counts with Activating Cell Count Feedback on Neuron Maturation Rate and Repressing Cell Count Feedback on Neuroblast Division Rate](#cellcountsneurondiffcellcountdownnbdiv)
- [Modeling Cell Counts with Activating Cell Count Feedback on Neuron Maturation Rate, Repressing Cell Count Feedback on Neuroblast Division Rate, and Repressing Cell Count Feedback on GMC Division Rate](#cellcountsneurondiffcellcountdownnbdivdowngmcdiv)

<a id="cellcounts"></a>
## Modeling Cell Counts

This is a simple ODE model of that tracks the counts of each cell type in the developing neuroblast lineage. This model does not track cell volume and assumes homogeneous behavior within each cell type.

### Model Species
- **$N_{\text{NB}}$** - Number of neuroblasts, neuroblasts are stem-like cells that can divide asymmetrically yielding one neuroblast and one GMC or symmetrically yielding two neuroblasts
- **$N_{\text{GMC}}$** - Number of GMCs, Ganglion Mother Cells are neuron precursors that divide symmetrically into two neurons
- **$N_{\text{ImNeuron}}$** - Number of immature neurons. Immature neurons are terminally differentiated brain cells that do not grow nor divide. they mature into MatNeurons
- **$N_{\text{MatNeuron}}$** - Number of mature neurons. Mature neurons are terminally differentiated brain cells that do not grow, divide, nor change state.

### Model Parameters
- **$k_{\text{NB}}$** - the rate of neuroblast divisions, in units of divisions/hour
- **$k_{\text{GMC}}$** - the rate of GMC divisions, in units of divisions/hour
- **$k_{\text{Neuron}}$** — Rate at which immature neurons mature into mature neurons (transitions/hour).
- **$\text{sym\_frac}$** - the fraction of neuroblast divisions that are symmetrical. Unitless.
  - Set to 0 for WT simulations and .15 for mudmut simulations

### Model Structure
The number of neuroblasts increases by 1 with each symmetric neuroblast division
$$\frac{dN_{\text{NB}}}{dt} = \text{sym\_frac} * k_{\text{NB}} * N_{\text{NB}}$$

The number of GMCs increases by 1 with each asymmetric neuroblast division and decreases by 1 with each GMC division.
$$\frac{dN_{\text{GMC}}}{dt} = (1 - \text{sym\_frac}) \cdot k_{\text{NB}} N_{\text{NB}} - k_{\text{GMC}} N_{\text{GMC}}$$

Immature neurons increase by 2 per GMC division and decrease as they mature:
$$
\frac{dN_{\text{ImNeuron}}}{dt} = 2 \cdot k_{\text{GMC}} \cdot N_{\text{GMC}} - k_{\text{Neuron}} \cdot N_{\text{ImNeuron}}
$$

Mature neurons increase as immature neurons mature:
$$
\frac{dN_{\text{MatNeuron}}}{dt} = k_{\text{Neuron}} \cdot N_{\text{ImNeuron}}
$$

### Model Limitations
- This model assumes homogeneous populations of neuroblasts, GMCs, and neurons, meaning that all cells within each type behave identically. It does not account for variability in growth rates, cell cycle timing, cell volume, or stochasticity in fate decisions
- This model treats the rates of division as constant over time and independent of population size or environmental feedback
- This model does not track cell volumes

In [ ]:
def neuroblast_model(t, y, params):
    N_NB, N_GMC, N_ImNeuron, N_MatNeuron = y
    k_NB, k_GMC, k_Neuron, sym_frac = params  # Division and maturation rates

    # Neuroblast divisions
    sym_divs = sym_frac * k_NB * N_NB
    asym_divs = (1 - sym_frac) * k_NB * N_NB

    # ODEs
    dN_NB = sym_divs
    dN_GMC = asym_divs - k_GMC * N_GMC
    dN_ImNeuron = 2 * k_GMC * N_GMC - k_Neuron * N_ImNeuron
    dN_MatNeuron = k_Neuron * N_ImNeuron

    return [dN_NB, dN_GMC, dN_ImNeuron, dN_MatNeuron]

### WT Simulation
**Parameter Justifications**
- Neuroblast division date - 1 division per 1.5 hours, from literature
- GMC division rate - 1 division per 9 hours, from literature
- Neuron maturation rate - 1 maturation per 48 hours, calibrated to allow WT lineages to have an average of 42 visable cells by the end of 48 hours

In [ ]:
# Parameter value assignment
k_NB = 1/1.5
k_GMC = 1/9.0
k_Neuron = 1/48

# Parameters: [k_NB, k_GMC, k_Neuron, sym_frac]
params = [
    k_NB,    # Neuroblast divides every 1.5 hrs
    k_GMC,   # GMC divides every 9 hrs
    k_Neuron,   # Immature neurons mature every 48 hrs
    0.0      # 0% symmetric NB divisions for WT
]

# Initial conditions: [N_NB, N_GMC, N_ImNeuron, N_MatNeuron]
y0 = [1, 0, 0, 0]

# Time span
t_span = [0, 48]  # simulate 48 hours
t_eval = np.linspace(*t_span, 500)

# Solve ODE
sol = solve_ivp(neuroblast_model, t_span, y0, t_eval=t_eval, args=(params,))

# Plotting
fig, ax = plt.subplots()

labels = ["Neuroblasts", "GMCs", "Immature Neurons", "Mature Neurons"]
y_data = [sol.y[0], sol.y[1], sol.y[2], sol.y[3]]

for y, label in zip(y_data, labels):
    ax.plot(sol.t, y, label=label)

ax.set_xlim(0, sol.t[-1] + 5)
ax.set_ylim(bottom=0)
annotate_final_values(ax, sol.t, y_data, labels)
ax.set_xlabel("Time (hours)")
ax.set_ylabel("Cell Count")
ax.set_title("WT lineage cell counts over 48 hours")
ax.grid()
plt.tight_layout()
plt.show()

### mudmut Simulation
**Parameter Justifications**
I left all the parameters the same as the WT simulation except changed the percent symmetric divisions to .15.

In [ ]:
# Parameter value assignment
k_NB = 1/1.5
k_GMC = 1/9.0
k_Neuron = 1/48

# Parameters: [k_NB, k_GMC, k_Neuron, sym_frac]
params = [
    k_NB,   # Neuroblast divides every 1.5 hrs
    k_GMC,   # GMC divides every 9 hrs
    k_Neuron,   # Immature neurons mature every 48 hrs
    0.15      # 15% symmetric NB divisions for mudmut
]

# Initial conditions: [N_NB, N_GMC, N_ImNeuron, N_MatNeuron]
y0 = [1, 0, 0, 0]

# Time span
t_span = [0, 48]  # simulate 48 hours
t_eval = np.linspace(*t_span, 500)

# Solve ODE
sol = solve_ivp(neuroblast_model, t_span, y0, t_eval=t_eval, args=(params,))

# Plotting
fig, ax = plt.subplots()

labels = ["Neuroblasts", "GMCs", "Immature Neurons", "Mature Neurons"]
y_data = [sol.y[0], sol.y[1], sol.y[2], sol.y[3]]

for y, label in zip(y_data, labels):
    ax.plot(sol.t, y, label=label)

ax.set_xlim(0, sol.t[-1] + 5)
ax.set_ylim(bottom=0)
annotate_final_values(ax, sol.t, y_data, labels)
ax.set_xlabel("Time (hours)")
ax.set_ylabel("Cell Count")
ax.set_title("mudmut lineage cell counts over 48 hours")
ax.grid()
plt.tight_layout()
plt.show()

### Conclusion from simple cell count model:
- ODE modling of cell growth and division behavior yields mud colonies with many more cells than wt colonies, which is the opposite of what is observed experimentally

[Back to outline](#outline)

---
<a id="cellcountsneurondiffcellcount"></a>
## Modeling cell counts with cellcount activating feedback on neuron maturation rate

This ODE model tracks the counts of each cell type in the developing neuroblast lineage and adds **activating feedback on the neuron maturation step** (immature → mature). The feedback depends on the **total number of cells** in the lineage and is modeled with a Hill activation function (setting \(n=1\) recovers a Michaelis–Menten form).

### Model Species
- **$N_{\text{NB}}$** — Number of neuroblasts (NB). Neuroblasts can divide asymmetrically (→ one NB + one GMC) or symmetrically (→ two NBs).
- **$N_{\text{GMC}}$** — Number of Ganglion Mother Cells (GMC). GMCs divide symmetrically into two immature neurons.
- **$N_{\text{ImNeuron}}$** — Number of immature neurons. These do not grow nor divide; they **mature** into mature neurons.
- **$N_{\text{MatNeuron}}$** — Number of mature neurons. Mature neurons do not grow, divide, or change state.

### Model Parameters
- **$k_{\text{NB}}$** — Neuroblast division rate (divisions/hour).
- **$k_{\text{GMC}}$** — GMC division rate (divisions/hour).
- **$k_{\text{Neuron,max}}$** — Maximum maturation rate from immature to mature neurons (transitions/hour).
- **$K_{\text{feedback}}$** — Half‑max constant (cell count) for maturation feedback based on total lineage size.
- **$n$** — Hill coefficient controlling the steepness of feedback (set \(n=1\) for Michaelis–Menten–like behavior).
- **$\text{sym\_frac}$** — Fraction of symmetric NB divisions (unitless).
  - Set to 0 for WT simulations and 0.15 for *mud* simulations.

### Model Structure

Let the total cell count be
$$
N_{\text{total}} \;=\; N_{\text{NB}} + N_{\text{GMC}} + N_{\text{ImNeuron}} + N_{\text{MatNeuron}}.
$$

Maturation is activate by total cell count via a Hill function:
$$
k_{\text{Neuron,eff}} \;=\; k_{\text{Neuron,max}} \cdot \frac{K_{\text{total}}^{\,n}}{K_{\text{feedback}}^{\,n} + N_{\text{total}}^{\,n}}.
$$

Divisions and maturation dynamics:
$$
\frac{dN_{\text{NB}}}{dt} \;=\; \text{sym\_frac}\cdot k_{\text{NB}}\, N_{\text{NB}},
$$
$$
\frac{dN_{\text{GMC}}}{dt} \;=\; (1-\text{sym\_frac})\cdot k_{\text{NB}}\, N_{\text{NB}} \;-\; k_{\text{GMC}}\, N_{\text{GMC}},
$$
$$
\frac{dN_{\text{ImNeuron}}}{dt} \;=\; 2\,k_{\text{GMC}}\, N_{\text{GMC}} \;-\; k_{\text{Neuron,eff}}\, N_{\text{ImNeuron}},
$$
$$
\frac{dN_{\text{MatNeuron}}}{dt} \;=\; k_{\text{Neuron,eff}}\, N_{\text{ImNeuron}}.
$$

### Model Limitations
- Assumes homogeneous behavior within each cell type (no cell‑to‑cell variability).
- Uses continuous, deterministic rates (no explicit event timing or stochasticity).
- Feedback is global and based only on **total** cell count (no spatial effects).

In [ ]:
def neuroblast_model_with_maturation_feedback(t, y, params):
    """
    ODEs for NB/GMC/neuron counts with Hill-type activation feedback on neuron maturation.

    State vector:
        y = [N_NB, N_GMC, N_ImNeuron, N_MatNeuron]

    Parameters (tuple/list in this order):
        k_NB            : NB division rate (1/hr)
        k_GMC           : GMC division rate (1/hr)
        k_Neuron_max    : maximum maturation rate (ImNeuron -> MatNeuron) (1/hr)
        K_feedback      : half-max total cell count for maturation feedback
        n               : Hill coefficient (n=1 -> Michaelis–Menten-like)
        sym_frac        : fraction of symmetric NB divisions (unitless)

    Feedback (activation) form:
        k_Neuron_eff = k_Neuron_max * (N_total^n) / (K_feedback^n + N_total^n)
        where N_total = N_NB + N_GMC + N_ImNeuron + N_MatNeuron
    """
    N_NB, N_GMC, N_ImNeuron, N_MatNeuron = y
    k_NB, k_GMC, k_Neuron_max, K_feedback, n, sym_frac = params

    # Total cells for feedback
    N_total = max(N_NB + N_GMC + N_ImNeuron + N_MatNeuron, 0.0)

    # Effective maturation rate with Hill-type repression
    if K_feedback <= 0:
        # Guard: if K_feedback is non-positive, fall back to no feedback
        k_Neuron_eff = k_Neuron_max
    else:
        k_Neuron_eff = k_Neuron_max * (max(N_total, 0.0)**n) / (K_feedback**n + max(N_total, 0.0)**n)

    # NB divisions
    sym_divs  = sym_frac * k_NB * N_NB
    asym_divs = (1.0 - sym_frac) * k_NB * N_NB

    # ODEs
    dN_NB        = sym_divs
    dN_GMC       = asym_divs - k_GMC * N_GMC
    dN_ImNeuron  = 2.0 * k_GMC * N_GMC - k_Neuron_eff * N_ImNeuron
    dN_MatNeuron = k_Neuron_eff * N_ImNeuron

    return [dN_NB, dN_GMC, dN_ImNeuron, dN_MatNeuron]

### WT Simulation
**Parameter Justifications**
- Cell count at which neurons reach half-max maturation rate: 41 - In an attempt to make the neuron maturation rate feedback rate as high as possible, I thought I'd set this value so that the number of cells in the lineage we expect to see at the last timepoint has the differentiation only half-maximally activated.
- Maximum neuron maturation rate: 1/24 hours - I set this to make the WT simulation have 41 visible cells at the last timepoint

In [ ]:
# WT simulation for maturation-activation feedback model

# --- Parameter value assignment ---
k_NB = 1/1.5          # NB divides every 1.5 hrs
k_GMC = 1/9.0         # GMC divides every 9 hrs
k_Neuron_max = 1/24   # Max maturation rate: ImNeuron -> MatNeuron in ~24 hrs
K_feedback = 41       # Half-max activation at ~20 total cells
n = 1                 # Hill coefficient (n=1 => Michaelis–Menten-like activation)
sym_frac = 0.0        # 0% symmetric NB divisions for WT

params = [k_NB, k_GMC, k_Neuron_max, K_feedback, n, sym_frac]

# --- Initial conditions ---
# y = [N_NB, N_GMC, N_ImNeuron, N_MatNeuron]
y0 = [1, 0, 0, 0]

# --- Time span ---
t_span = [0, 48]                          # simulate 48 hours
t_eval = np.linspace(*t_span, 500)

# --- Solve ODE ---
sol = solve_ivp(
    neuroblast_model_with_maturation_feedback,
    t_span, y0, t_eval=t_eval, args=(params,)
)

# --- Plotting: cell counts ---
fig, ax = plt.subplots()
labels = ["Neuroblasts", "GMCs", "Immature Neurons", "Mature Neurons"]
y_data = [sol.y[0], sol.y[1], sol.y[2], sol.y[3]]

for y, label in zip(y_data, labels):
    ax.plot(sol.t, y, label=label)

ax.set_xlim(0, sol.t[-1] + 5)
ax.set_ylim(bottom=0)
# Uses your existing helper; comment out if not defined in your notebook
annotate_final_values(ax, sol.t, y_data, labels)

ax.set_xlabel("Time (hours)")
ax.set_ylabel("Cell Count")
ax.set_title("WT lineage cell counts over 48 hours (maturation activation feedback)")
ax.grid()
ax.legend()
plt.tight_layout()
plt.show()

# --- plot effective maturation rate over time to verify feedback behavior ---
# Compute k_Neuron_eff(t) for reference
N_total = sol.y[0] + sol.y[1] + sol.y[2] + sol.y[3]
k_eff = k_Neuron_max * (N_total**n) / (K_feedback**n + N_total**n)

fig, ax = plt.subplots()
ax.plot(sol.t, k_eff)
ax.set_xlabel("Time (hours)")
ax.set_ylabel("Effective maturation rate  k_Neuron_eff  (1/hr)")
ax.set_title("Effective maturation rate under activation feedback")
ax.grid()
plt.tight_layout()
plt.show()

### mudmut Simulation
**Parameter Justification**
- fraction of symmetrical divisions: .15 - capturing mudmut dynamics

In [ ]:
# mudmut simulation for maturation-activation feedback model

# --- Parameter value assignment ---
k_NB = 1/1.5          # NB divides every 1.5 hrs
k_GMC = 1/9.0         # GMC divides every 9 hrs
k_Neuron_max = 1/24   # Max maturation rate: ImNeuron -> MatNeuron in ~24 hrs
K_feedback = 41       # Half-max activation at ~20 total cells
n = 1                 # Hill coefficient (n=1 => Michaelis–Menten-like activation)
sym_frac = 0.15       # 15% symmetric NB divisions for mudmut

params = [k_NB, k_GMC, k_Neuron_max, K_feedback, n, sym_frac]

# --- Initial conditions ---
# y = [N_NB, N_GMC, N_ImNeuron, N_MatNeuron]
y0 = [1, 0, 0, 0]

# --- Time span ---
t_span = [0, 48]                          # simulate 48 hours
t_eval = np.linspace(*t_span, 500)

# --- Solve ODE ---
sol = solve_ivp(
    neuroblast_model_with_maturation_feedback,
    t_span, y0, t_eval=t_eval, args=(params,)
)

# --- Plotting: cell counts ---
fig, ax = plt.subplots()
labels = ["Neuroblasts", "GMCs", "Immature Neurons", "Mature Neurons"]
y_data = [sol.y[0], sol.y[1], sol.y[2], sol.y[3]]

for y, label in zip(y_data, labels):
    ax.plot(sol.t, y, label=label)

ax.set_xlim(0, sol.t[-1] + 5)
ax.set_ylim(bottom=0)
# Uses your existing helper; comment out if not defined in your notebook
annotate_final_values(ax, sol.t, y_data, labels)

ax.set_xlabel("Time (hours)")
ax.set_ylabel("Cell Count")
ax.set_title("mudmut lineage cell counts over 48 hours (maturation activation feedback)")
ax.grid()
ax.legend()
plt.tight_layout()
plt.show()

# --- plot effective maturation rate over time to verify feedback behavior ---
# Compute k_Neuron_eff(t) for reference
N_total = sol.y[0] + sol.y[1] + sol.y[2] + sol.y[3]
k_eff = k_Neuron_max * (N_total**n) / (K_feedback**n + N_total**n)

fig, ax = plt.subplots()
ax.plot(sol.t, k_eff)
ax.set_xlabel("Time (hours)")
ax.set_ylabel("Effective maturation rate  k_Neuron_eff  (1/hr)")
ax.set_title("Effective maturation rate under activation feedback")
ax.grid()
plt.tight_layout()
plt.show()

### Conclusions:
- Still insufficient feedback

[Back to outline](#outline)

---
<a id="cellcountsneurondiffcellcountdownnbdiv"></a>

## Cell-Count–Coupled Model: Neuron Maturation Activation + NB Division Repression

This model extends the count-only lineage ODEs by coupling two processes to **total lineage size** $N_{\text{total}}$:
- **Activation** of immature→mature neuron maturation as $N_{\text{total}}$ increases.
- **Repression** of neuroblast (NB) division as $N_{\text{total}}$ increases.

This lets the lineage **accelerate differentiation** while **slowing stem-like expansion** as it becomes larger.

### Model Species
- **$N_{\text{NB}}$** — Neuroblasts.
- **$N_{\text{GMC}}$** — Ganglion Mother Cells.
- **$N_{\text{ImNeuron}}$** — Immature neurons.
- **$N_{\text{MatNeuron}}$** — Mature neurons.

### Model Parameters
- **$k_{\text{NB,max}}$** — Max NB division rate (1/hr).
- **$k_{\text{GMC}}$** — GMC division rate (1/hr).
- **$k_{\text{Neuron,max}}$** — Max maturation rate (1/hr).
- **$K_{\text{div}},\, n_{\text{div}}$** — Half-max constant & Hill coefficient for **NB division repression**.
- **$K_{\text{mat}},\, n_{\text{mat}}$** — Half-max constant & Hill coefficient for **maturation activation**.
- **$\text{sym\_frac}$** — Fraction of symmetric NB divisions (unitless).

### Feedback Forms (with $N_{\text{total}}=N_{\text{NB}}+N_{\text{GMC}}+N_{\text{ImNeuron}}+N_{\text{MatNeuron}}$)
**NB division (repression):**
$$
k_{\text{NB,eff}} \;=\; k_{\text{NB,max}}\;\frac{K_{\text{div}}^{\,n_{\text{div}}}}{K_{\text{div}}^{\,n_{\text{div}}} + N_{\text{total}}^{\,n_{\text{div}}}}
$$

**Neuron maturation (activation):**
$$
k_{\text{Neuron,eff}} \;=\; k_{\text{Neuron,max}}\;\frac{N_{\text{total}}^{\,n_{\text{mat}}}}{K_{\text{mat}}^{\,n_{\text{mat}}} + N_{\text{total}}^{\,n_{\text{mat}}}}
$$

### ODEs
Let $text{sym\_frac}\in[0,1]$, then per-unit-time symmetric and asymmetric NB divisions are:
$$
\text{sym\_divs}= \text{sym\_frac}\;k_{\text{NB,eff}}\,N_{\text{NB}},\qquad
\text{asym\_divs}= (1-\text{sym\_frac})\;k_{\text{NB,eff}}\,N_{\text{NB}}.
$$
Population dynamics:
$$
\frac{dN_{\text{NB}}}{dt} = \text{sym\_divs}, \qquad
\frac{dN_{\text{GMC}}}{dt} = \text{asym\_divs} - k_{\text{GMC}}\,N_{\text{GMC}},
$$
$$
\frac{dN_{\text{ImNeuron}}}{dt} = 2\,k_{\text{GMC}}\,N_{\text{GMC}} - k_{\text{Neuron,eff}}\,N_{\text{ImNeuron}},\qquad
\frac{dN_{\text{MatNeuron}}}{dt} = k_{\text{Neuron,eff}}\,N_{\text{ImNeuron}}.
$$

### Notes
- Set $n_{\text{div}}=1$ or $n_{\text{mat}}=1$ to recover Michaelis–Menten–like responses; larger \(n\) yields more switch-like behavior.
- This is a **count-only** model (no volume tracking).

In [ ]:
# Model: maturation activation + NB division repression by total cell count
def neuroblast_model_maturation_activation_and_division_repression(t, y, params):
    """
    y = [N_NB, N_GMC, N_ImNeuron, N_MatNeuron]
    params = [
        k_NB_max,       # max NB division rate (1/hr)
        k_GMC,          # GMC division rate (1/hr)
        k_Neuron_max,   # max maturation rate (1/hr)
        K_div, n_div,   # repression params for NB division
        K_mat, n_mat,   # activation params for maturation
        sym_frac        # fraction of symmetric NB divisions
    ]
    """
    N_NB, N_GMC, N_Im, N_Mat = y
    (k_NB_max, k_GMC, k_Neuron_max,
     K_div, n_div, K_mat, n_mat,
     sym_frac) = params

    # Total cells for feedback
    N_total = max(N_NB + N_GMC + N_Im + N_Mat, 0.0)

    # NB division repression (Hill-type)
    if K_div <= 0:
        k_NB_eff = k_NB_max
    else:
        k_NB_eff = k_NB_max * (K_div**n_div) / (K_div**n_div + N_total**n_div)

    # Neuron maturation activation (Hill-type)
    if K_mat <= 0:
        k_Neuron_eff = k_Neuron_max
    else:
        k_Neuron_eff = k_Neuron_max * (N_total**n_mat) / (K_mat**n_mat + N_total**n_mat)

    # Division flows
    sym_divs  = sym_frac * k_NB_eff * N_NB
    asym_divs = (1.0 - sym_frac) * k_NB_eff * N_NB

    # ODEs
    dN_NB        = sym_divs
    dN_GMC       = asym_divs - k_GMC * N_GMC
    dN_Im        = 2.0 * k_GMC * N_GMC - k_Neuron_eff * N_Im
    dN_Mat       = k_Neuron_eff * N_Im

    return [dN_NB, dN_GMC, dN_Im, dN_Mat]

### WT Simulation
**Parameter Justifications**
- cell count at which neuron division repression is half-maximal : 41 - making it the same as the half-max activation of neuron maturation to start
- max neuron division rate: 60 mins - calibrated to the quarter hour that got the output closest to 41 visable cells at the last time point

In [ ]:
# --- WT simulation cell ---

# Baseline parameters (tweak K_div/K_mat and n's to explore behaviors)
k_NB_max     = 1 / 1    # NB could divide every 60 mins at small N_total
k_GMC        = 1 / 9.0    # GMC divides every 9 h
k_Neuron_max = 1 / 24.0   # Max maturation ~24 h
K_div, n_div = 41.0, 1    # repression half-max & Hill coeff for NB division
K_mat, n_mat = 41.0, 1    # activation half-max & Hill coeff for maturation
sym_frac     = 0.0        # WT asymmetric divisions

params = [k_NB_max, k_GMC, k_Neuron_max, K_div, n_div, K_mat, n_mat, sym_frac]

# Initial conditions and integration
y0 = [1, 0, 0, 0]      # start with one NB
t_span = [0, 48]
t_eval = np.linspace(*t_span, 500)

sol = solve_ivp(
    neuroblast_model_maturation_activation_and_division_repression,
    t_span, y0, t_eval=t_eval, args=(params,),
    # method='LSODA'  # uncomment if you encounter stiffness
)

# Plot cell counts
fig, ax = plt.subplots()
labels = ["Neuroblasts", "GMCs", "Immature Neurons", "Mature Neurons"]
series = [sol.y[0], sol.y[1], sol.y[2], sol.y[3]]

for s, label in zip(series, labels):
    ax.plot(sol.t, s, label=label)

ax.set_xlim(0, sol.t[-1] + 5)
ax.set_ylim(bottom=0)
try:
    annotate_final_values(ax, sol.t, series, labels)  # if helper is defined in your notebook
except NameError:
    pass
ax.set_xlabel("Time (hours)")
ax.set_ylabel("Cell Count")
ax.set_title("WT lineage: maturation activation + NB division repression")
ax.grid()
ax.legend()
plt.tight_layout()
plt.show()

# visualize effective rates over time
N_total = sol.y[0] + sol.y[1] + sol.y[2] + sol.y[3]
k_NB_eff = k_NB_max * (K_div**n_div) / (K_div**n_div + N_total**n_div)
k_Neuron_eff = k_Neuron_max * (N_total**n_mat) / (K_mat**n_mat + N_total**n_mat)

fig, ax = plt.subplots()
ax.plot(sol.t, k_NB_eff, label="k_NB_eff (repressed)")
ax.plot(sol.t, k_Neuron_eff, label="k_Neuron_eff (activated)")
ax.set_xlabel("Time (hours)")
ax.set_ylabel("Rate (1/hr)")
ax.set_title("Effective rates under combined feedback")
ax.grid()
ax.legend()
plt.tight_layout()
plt.show()

### mudmut Simulation

In [ ]:
# --- WT simulation cell ---

# Baseline parameters (tweak K_div/K_mat and n's to explore behaviors)
k_NB_max     = 1 / 1    # NB could divide every 60 mins at small N_total
k_GMC        = 1 / 9.0    # GMC divides every 9 h
k_Neuron_max = 1 / 24.0   # Max maturation ~24 h
K_div, n_div = 41.0, 1    # repression half-max & Hill coeff for NB division
K_mat, n_mat = 41.0, 1    # activation half-max & Hill coeff for maturation
sym_frac     = 0.15        # WT asymmetric divisions

params = [k_NB_max, k_GMC, k_Neuron_max, K_div, n_div, K_mat, n_mat, sym_frac]

# Initial conditions and integration
y0 = [1, 0, 0, 0]      # start with one NB
t_span = [0, 48]
t_eval = np.linspace(*t_span, 500)

sol = solve_ivp(
    neuroblast_model_maturation_activation_and_division_repression,
    t_span, y0, t_eval=t_eval, args=(params,),
    # method='LSODA'  # uncomment if you encounter stiffness
)

# Plot cell counts
fig, ax = plt.subplots()
labels = ["Neuroblasts", "GMCs", "Immature Neurons", "Mature Neurons"]
series = [sol.y[0], sol.y[1], sol.y[2], sol.y[3]]

for s, label in zip(series, labels):
    ax.plot(sol.t, s, label=label)

ax.set_xlim(0, sol.t[-1] + 5)
ax.set_ylim(bottom=0)
try:
    annotate_final_values(ax, sol.t, series, labels)  # if helper is defined in your notebook
except NameError:
    pass
ax.set_xlabel("Time (hours)")
ax.set_ylabel("Cell Count")
ax.set_title("mudmut lineage: maturation activation + NB division repression")
ax.grid()
ax.legend()
plt.tight_layout()
plt.show()

# visualize effective rates over time
N_total = sol.y[0] + sol.y[1] + sol.y[2] + sol.y[3]
k_NB_eff = k_NB_max * (K_div**n_div) / (K_div**n_div + N_total**n_div)
k_Neuron_eff = k_Neuron_max * (N_total**n_mat) / (K_mat**n_mat + N_total**n_mat)

fig, ax = plt.subplots()
ax.plot(sol.t, k_NB_eff, label="k_NB_eff (repressed)")
ax.plot(sol.t, k_Neuron_eff, label="k_Neuron_eff (activated)")
ax.set_xlabel("Time (hours)")
ax.set_ylabel("Rate (1/hr)")
ax.set_title("Effective rates under combined feedback")
ax.grid()
ax.legend()
plt.tight_layout()
plt.show()

[Back to outline](#outline)

---
<a id="cellcountsneurondiffcellcountdownnbdivdowngmcdiv"></a>

## Cell-Count–Coupled Model: NB Division Repression + GMC Division Repression + Neuron Maturation Activation

This model extends the count-only neuroblast lineage ODEs by coupling three processes to **total lineage size** $N_{\text{total}}$:

- **Repression** of **neuroblast (NB) division** as $N_{\text{total}}$ increases.
- **Repression** of **ganglion mother cell (GMC) division** as $N_{\text{total}}$ increases.
- **Activation** of **immature→mature neuron maturation** as $N_{\text{total}}$ increases.

This formulation mimics a scenario where larger lineages slow their expansion (both NB and GMC proliferation) while accelerating terminal differentiation.

### Model Species
- **$N_{\text{NB}}$** — Neuroblasts.
- **$N_{\text{GMC}}$** — Ganglion Mother Cells.
- **$N_{\text{ImNeuron}}$** — Immature neurons.
- **$N_{\text{MatNeuron}}$** — Mature neurons.

### Model Parameters
- **$k_{\text{NB,max}}$** — Max NB division rate (1/hr).
- **$k_{\text{GMC,max}}$** — Max GMC division rate (1/hr).
- **$k_{\text{Neuron,max}}$** — Max maturation rate (1/hr).
- **$K_{\text{div,NB}}, n_{\text{div,NB}}$** — Half-max constant & Hill coefficient for **NB division repression**.
- **$K_{\text{div,GMC}}, n_{\text{div,GMC}}$** — Half-max constant & Hill coefficient for **GMC division repression**.
- **$K_{\text{mat}}, n_{\text{mat}}$** — Half-max constant & Hill coefficient for **maturation activation**.
- **$\text{sym\_frac}$** — Fraction of symmetric NB divisions (unitless).

### Feedback Forms
Let $$N_{\text{total}}=N_{\text{NB}}+N_{\text{GMC}}+N_{\text{ImNeuron}}+N_{\text{MatNeuron}}$$.

**NB division (repression):**
$$
k_{\text{NB,eff}} = k_{\text{NB,max}} \cdot \frac{K_{\text{div,NB}}^{n_{\text{div,NB}}}}{K_{\text{div,NB}}^{n_{\text{div,NB}}} + N_{\text{total}}^{n_{\text{div,NB}}}}
$$

**GMC division (repression):**
$$
k_{\text{GMC,eff}} = k_{\text{GMC,max}} \cdot \frac{K_{\text{div,GMC}}^{n_{\text{div,GMC}}}}{K_{\text{div,GMC}}^{n_{\text{div,GMC}}} + N_{\text{total}}^{n_{\text{div,GMC}}}}
$$

**Neuron maturation (activation):**
$$
k_{\text{Neuron,eff}} = k_{\text{Neuron,max}} \cdot \frac{N_{\text{total}}^{n_{\text{mat}}}}{K_{\text{mat}}^{n_{\text{mat}}} + N_{\text{total}}^{n_{\text{mat}}}}
$$

### ODEs
With \(\text{sym\_frac} \in [0,1]\):
$$
\text{sym\_divs} = \text{sym\_frac} \cdot k_{\text{NB,eff}} \cdot N_{\text{NB}}, \quad
\text{asym\_divs} = (1 - \text{sym\_frac}) \cdot k_{\text{NB,eff}} \cdot N_{\text{NB}}.
$$

$$
\frac{dN_{\text{NB}}}{dt} = \text{sym\_divs}
$$
$$
\frac{dN_{\text{GMC}}}{dt} = \text{asym\_divs} - k_{\text{GMC,eff}} \cdot N_{\text{GMC}}
$$
$$
\frac{dN_{\text{ImNeuron}}}{dt} = 2 \cdot k_{\text{GMC,eff}} \cdot N_{\text{GMC}} - k_{\text{Neuron,eff}} \cdot N_{\text{ImNeuron}}
$$
$$
\frac{dN_{\text{MatNeuron}}}{dt} = k_{\text{Neuron,eff}} \cdot N_{\text{ImNeuron}}
$$

### Notes
- Set $n=1$ for Michaelis–Menten–like behavior or larger for more switch-like feedback.
- This is a **count-only** model (no volume tracking).

In [ ]:
# Model: repression on NB and GMC division, activation on neuron maturation
def neuroblast_model_double_repression_maturation_activation(t, y, params):
    """
    y = [N_NB, N_GMC, N_ImNeuron, N_MatNeuron]
    params = [
        k_NB_max,        # max NB division rate (1/hr)
        k_GMC_max,       # max GMC division rate (1/hr)
        k_Neuron_max,    # max maturation rate (1/hr)
        K_div_NB, n_div_NB,     # NB division repression params
        K_div_GMC, n_div_GMC,   # GMC division repression params
        K_mat, n_mat,           # maturation activation params
        sym_frac                # fraction of symmetric NB divisions
    ]
    """
    N_NB, N_GMC, N_Im, N_Mat = y
    (k_NB_max, k_GMC_max, k_Neuron_max,
     K_div_NB, n_div_NB, K_div_GMC, n_div_GMC,
     K_mat, n_mat,
     sym_frac) = params

    # Total cells for feedback
    N_total = max(N_NB + N_GMC + N_Im + N_Mat, 0.0)

    # NB division repression
    if K_div_NB <= 0:
        k_NB_eff = k_NB_max
    else:
        k_NB_eff = k_NB_max * (K_div_NB**n_div_NB) / (K_div_NB**n_div_NB + N_total**n_div_NB)

    # GMC division repression
    if K_div_GMC <= 0:
        k_GMC_eff = k_GMC_max
    else:
        k_GMC_eff = k_GMC_max * (K_div_GMC**n_div_GMC) / (K_div_GMC**n_div_GMC + N_total**n_div_GMC)

    # Neuron maturation activation
    if K_mat <= 0:
        k_Neuron_eff = k_Neuron_max
    else:
        k_Neuron_eff = k_Neuron_max * (N_total**n_mat) / (K_mat**n_mat + N_total**n_mat)

    # Division flows
    sym_divs  = sym_frac * k_NB_eff * N_NB
    asym_divs = (1.0 - sym_frac) * k_NB_eff * N_NB

    # ODEs
    dN_NB  = sym_divs
    dN_GMC = asym_divs - k_GMC_eff * N_GMC
    dN_Im  = 2.0 * k_GMC_eff * N_GMC - k_Neuron_eff * N_Im
    dN_Mat = k_Neuron_eff * N_Im

    return [dN_NB, dN_GMC, dN_Im, dN_Mat]

### WT Simulation
**Parameter justifictions**
- cell count at which GMC repression is half-maximal : 41 - kept this consistent with other cell-count half-maximal thresholds
- max GMC division rate : 1/8 hours - I set this to a whole number less than 9 (which we expect to be the average) that yielded cell counts near 41

In [ ]:
# --- WT simulation cell ---

# Parameters
k_NB_max   = 1 / 1      # max NB division rate 1 per hr
k_GMC_max  = 1 / 8.0    # max GMC division rate
k_Neuron_max = 1 / 24.0 # max maturation rate 1 per 24 hours

# Feedback params
K_div_NB, n_div_NB   = 41.0, 1
K_div_GMC, n_div_GMC = 41.0, 1
K_mat, n_mat         = 41.0, 1

sym_frac = 0.0 # WT

params = [
    k_NB_max, k_GMC_max, k_Neuron_max,
    K_div_NB, n_div_NB,
    K_div_GMC, n_div_GMC,
    K_mat, n_mat,
    sym_frac
]

# Initial conditions
y0 = [1, 0, 0, 0]  # start with one NB
t_span = [0, 48]
t_eval = np.linspace(*t_span, 500)

# Solve
sol = solve_ivp(
    neuroblast_model_double_repression_maturation_activation,
    t_span, y0, t_eval=t_eval, args=(params,)
)

# Plot cell counts
fig, ax = plt.subplots()
labels = ["Neuroblasts", "GMCs", "Immature Neurons", "Mature Neurons"]
series = [sol.y[0], sol.y[1], sol.y[2], sol.y[3]]

for s, label in zip(series, labels):
    ax.plot(sol.t, s, label=label)

ax.set_xlim(0, sol.t[-1] + 5)
ax.set_ylim(bottom=0)
try:
    annotate_final_values(ax, sol.t, series, labels)
except NameError:
    pass
ax.set_xlabel("Time (hours)")
ax.set_ylabel("Cell Count")
ax.set_title("WT: NB & GMC division repression + neuron maturation activation")
ax.grid()
ax.legend()
plt.tight_layout()
plt.show()

### mudmut simulation

In [ ]:
# --- WT simulation cell ---

# Parameters
k_NB_max   = 1 / 1      # max NB division rate 1 per hr
k_GMC_max  = 1 / 8.0    # max GMC division rate
k_Neuron_max = 1 / 24.0 # max maturation rate 1 per 24 hours

# Feedback params
K_div_NB, n_div_NB   = 41.0, 1
K_div_GMC, n_div_GMC = 41.0, 1
K_mat, n_mat         = 41.0, 1

sym_frac = 0.15 # mudmut

params = [
    k_NB_max, k_GMC_max, k_Neuron_max,
    K_div_NB, n_div_NB,
    K_div_GMC, n_div_GMC,
    K_mat, n_mat,
    sym_frac
]

# Initial conditions
y0 = [1, 0, 0, 0]  # start with one NB
t_span = [0, 48]
t_eval = np.linspace(*t_span, 500)

# Solve
sol = solve_ivp(
    neuroblast_model_double_repression_maturation_activation,
    t_span, y0, t_eval=t_eval, args=(params,)
)

# Plot cell counts
fig, ax = plt.subplots()
labels = ["Neuroblasts", "GMCs", "Immature Neurons", "Mature Neurons"]
series = [sol.y[0], sol.y[1], sol.y[2], sol.y[3]]

for s, label in zip(series, labels):
    ax.plot(sol.t, s, label=label)

ax.set_xlim(0, sol.t[-1] + 5)
ax.set_ylim(bottom=0)
try:
    annotate_final_values(ax, sol.t, series, labels)
except NameError:
    pass
ax.set_xlabel("Time (hours)")
ax.set_ylabel("Cell Count")
ax.set_title("mudmut: NB & GMC division repression + neuron maturation activation")
ax.grid()
ax.legend()
plt.tight_layout()
plt.show()